## Advection-Diffusion


#### Problem Setup

\begin{equation} 
0 = au_x + bu_y + cu_{xx} + du_{yy} - u_t \; \; \; \; \; \; \; \; \; (*)
\end{equation}

For the generation of our training data we use the same method as in PDE-Net in Tensorflow (for details please refer to that chapter in the paper) with:

$a = b = 2$ <br> $c = d = 0.5$ <br>

$u: \mathbb{R}^3 \rightarrow \mathbb{R}$ is some solution to $(*)$. <br>
$f: \mathbb{R}^3 \rightarrow \mathbb{R}, \;f(x,y,t) = 0$ <br>
$X_i := (x_i, y_i, t_i) \in [0,1] \times [0,1] \times [0, 0.135] \subseteq \mathbb{R}^3$ for $i \in \{1, \dotsc, n\}$

and our known function values will be $\{u(X_i), f(X_i)\}_{i \in \{1, \dotsc, n\}}$.

We assume that $u$ can be represented as a Gaussian process with SE kernel.

$u \sim \mathcal{GP}(0, k_{uu}(X_i, X_j; \theta))$, where $\theta = \{\sigma, l_x, l_y, l_t\}$.

Set the linear operator to:

$\mathcal{L}_X^{\phi} := a\partial_x + b\partial_y + c\partial_{x,x} + d\partial_{y,y} - \partial_t$

so that

$\mathcal{L}_X^{\phi} u = f = 0$

Problem at hand: Estimate $\phi:=\{a\}$ with $\{b,c,d\} = \{2, 0.5, 0.5\}$ fixed.


#### Step 1: Simulate data

In [1]:
import time
import numpy as np
import sympy as sp
import numpy.fft as fft
from scipy.linalg import solve_triangular
import scipy.optimize as opt

In [2]:
# Global variables: x, y, t, n, y_u, y_f, s

# Number of data samples
n = 80

# Noise of our data:
s = 1e-7

In [3]:
options = {'mesh_size': [250, 250],     # How large is the (regular) 2D-grid of function values for a fixed t.
                                        # Keep mesh_size[0] = mesh_size[1]
           'layers': 9,                 # Layers of the NN. Also counting the initial layer!
           'dt': 0.0015,                # Time discretization. We step dt*(layers - 1) forward in time.
           'batch_size': 1,             # We take a batch of sub-grids in space
           'noise_level': 0.0,          # Can add some noise to the data (not taken 1 to 1, gets multiplied by stddev)
           'downsample_by': 1,          # Size of sub-grids (in space) * downsample_by = mesh_size
           }

In [4]:
def initgen(mesh_size, freq=3, boundary='Periodic'):
    """
    Returns function values for t=0 on a regular grid of size 'mesh_size' in [0, 2*pi]x[0, 2*pi] as a matrix
    """
    # Default: (mesh_size, freq, boundary) = ([250, 250], 4, 'Periodic')
    if np.iterable(freq):
        return freq
    # 250x250 normally distributed variables IFFTed and FFTed:
    x = _initgen_periodic(mesh_size, freq=freq)
    x = x * 100
    if boundary.upper() == 'DIRICHLET':
        dim = x.ndim
        for i in range(dim):
            y = np.arange(mesh_size[i]) / mesh_size[i]
            y = y * (1 - y)
            s = np.ones(dim, dtype=np.int32)
            s[i] = mesh_size[i]
            y = np.reshape(y, s)
            x = x * y
        x = x[[slice(1, None), ] * dim]
        x = x * 16
    return x

In [5]:
def _initgen_periodic(mesh_size, freq=3):
    # np.random.seed(50)
    # Default: (mesh_size, freq) = ([250, 250], 4)
    dim = len(mesh_size)
    # Default: 250x250-matrix of normally distributed variables
    x = np.random.randn(*mesh_size)
    coe = fft.ifftn(x)
    # set frequency of generated initial value
    # Array of random ints in [freq, 2*freq - 1]
    freqs = np.random.randint(freq, 2 * freq, size=[dim, ])
    # freqs = [10,10]
    for i in range(dim):
        perm = np.arange(dim, dtype=np.int32)
        perm[i] = 0
        perm[0] = i
        # Permutes for i = 1 and does nothing for i = 0.
        coe = coe.transpose(*perm)
        coe[freqs[i] + 1:-freqs[i]] = 0
        coe = coe.transpose(*perm)
    x = fft.fftn(coe)
    assert np.linalg.norm(x.imag) < 1e-8
    x = x.real
    return x

In [6]:
def pad_input_2(input, pad_by):
    """
    We increase the size of input for all j by pad_by on each side of the matrix
    by inserting values from the opposite side
    """
    mesh_size = input.shape[0]

    B = np.eye(mesh_size, dtype=np.float32)
    for i in range(pad_by):
        a = np.zeros(mesh_size, dtype=np.float32)
        a[mesh_size - i - 1] = 1
        B = np.concatenate(([a], B), axis=0)
    for i in range(pad_by):
        a = np.zeros(mesh_size, dtype=np.float32)
        a[i] = 1
        B = np.concatenate((B, [a]), axis=0)

    return B @ input @ B.T

def downsample(sample, scale):
    """
    Returns a regular somewhat random sub-grid of sample, whose size is reduced by a factor of 'scale'.
    """
    # np.random.seed(50)
    idx1 = slice(np.random.randint(scale), None, scale)
    idx2 = slice(np.random.randint(scale), None, scale)
    # idx1 = slice(1, None, scale)
    # idx2 = slice(0, None, scale)

    for kwarg in sample:
        sample[kwarg] = sample[kwarg][idx1, idx2]
    return sample

def addNoise(sample, noise, layers):
    # Adding noise to u0
    mean = sample['u0'].mean()
    stdvar = np.sqrt(((sample['u0'] - mean) ** 2).mean())
    size = sample['u0'].shape
    startnoise = np.random.standard_normal(size)
    sample['u0'] = sample['u0'] + noise * stdvar * startnoise

    # Adding noise to ut, t > 0
    for l in range(1, layers):
        arg = 'u' + str(l)
        size = sample[arg].shape
        endnoise = np.random.standard_normal(size)
        sample[arg] = sample[arg] + noise * stdvar * endnoise

    return sample

In [7]:
def generate(options):
    """
    Generating data / function-values on a regular grid of space-time, adding noise and taking a batch of
    down-sampled regular sub-grids of this grid. This batch will contain the samples to train our network with.

    :param options: The dictionary of user-specified options (cf. main.py). Contains e.g. the grid-dimensions and noise
    :return: A batch (as a list) of samples (as dictionaries), that in turn consist of (noisy) function values on
             down-sampled sub-grids for all dt-layers.
    """
    # u_t = a*u_x + b*u_y + c*u_{xx} + d*u_{yy}

    # Variable declarations
    nx = options['mesh_size'][0]
    ny = options['mesh_size'][1]
    nt = options['layers']
    batch_size = options['batch_size']
    dt = options['dt']
    noise_level = options['noise_level']
    downsample_by = options['downsample_by']

    # Really good with a = b = 2
    a = 2
    b = 2
    c = 0.5
    d = 0.5

    dx = 2 * np.pi / (nx - 1)
    dy = 2 * np.pi / (ny - 1)

    ## Needed for plotting:
    # x = np.linspace(0, 2*np.pi, num = nx)
    # y = np.linspace(0, 2*np.pi, num = ny)
    # X, Y = np.meshgrid(x, y)

    batch = []

    for i in range(batch_size):
        ############ Change the following lines to implement your own data ############

        # Assign initial function:
        u = initgen(options['mesh_size'], freq=4, boundary='Periodic')

        ## Plotting the initial function:
        # fig = plt.figure(figsize=(11,7), dpi=100)
        # ax = fig.gca(projection='3d')
        # surf = ax.plot_surface(X, Y, u[:], cmap=cm.viridis)
        #
        # plt.show()

        sample = {}
        sample['u0'] = u

        for n in range(nt - 1):
            un = pad_input_2(u, 2)[1:, 1:]

            u = (un[1:-1, 1:-1] + c * dt / dx ** 2 * (un[1:-1, 2:] - 2 * un[1:-1, 1:-1] + un[1:-1, 0:-2])
                 + d * dt / dy ** 2 * (un[2:, 1: -1] - 2 * un[1:-1, 1:-1] + un[0:-2, 1:-1])
                 + a * dt / dx * (un[1:-1, 2:] - un[1:-1, 1:-1])
                 + b * dt / dy * (un[2:, 1:-1] - un[1:-1, 1:-1]))[:-1, :-1]

            sample['u' + str(n + 1)] = u

        ## sample should at this point be a dictionary with entries 'u0', ..., 'uL', where L = nt                   ##
        ## For a given j, sample['uj'] is a matrix of size nx x ny containing the function values at time-step dt*j ##
        ##############################################################################################################

        ## Plotting the function values from the last layer:
        # fig2 = plt.figure()
        # ax2 = fig2.gca(projection='3d')
        # surf2 = ax2.plot_surface(X, Y, u, cmap=cm.viridis)
        #
        # plt.show()

        downsample(sample, downsample_by)
        addNoise(sample, noise_level, nt)

        batch.append(sample)

    return batch

Note, that we only take $n$ data samples with their corresponding function values from the grid.

In [8]:
def simulate_data():
    batch = generate(options)
    
    x_val_arr = []
    y_val_arr = []
    t_val_arr = []
    val_arr = []
    
    for i in range(n): 
        t_rand = np.random.randint(options['layers'])
        # Data is in [0,1]^2:
        x_rand = np.random.randint(options['mesh_size'][0]//(2*np.pi))
        y_rand = np.random.randint(options['mesh_size'][1]//(2*np.pi))

        val = batch[0]['u' + str(t_rand)][x_rand, y_rand]

        x_val = 2 * np.pi / (options['mesh_size'][0] - 1) * x_rand
        y_val = 2 * np.pi / (options['mesh_size'][1] - 1) * y_rand
        t_val = t_rand * options['dt']
        
        x_val_arr.append(x_val)
        y_val_arr.append(y_val)
        t_val_arr.append(t_val)
        val_arr.append(val)
        
    return (np.array(x_val_arr), np.array(y_val_arr), np.array(t_val_arr), np.array(val_arr), np.zeros(len(val_arr)))
(x,y,t,y_u,y_f) = simulate_data()

#### Step 2: Evaluate kernels

$k_{uu}(X_i, X_j; \theta) = \sigma \cdot exp(-\frac{1}{2l_x}(x_i-x_j)^2 - \frac{1}{2l_y}(y_i-y_j)^2 - \frac{1}{2l_t}(t_i-t_j)^2)$

In [9]:
x_i, x_j, y_i, y_j, t_i, t_j, sigma, l_x, l_y, l_t, a, b, c, d = sp.symbols('x_i x_j y_i y_j t_i t_j sigma l_x l_y l_t a b c d')
kuu_sym = sigma*sp.exp(-1/(2*l_x)*((x_i - x_j)**2) - 1/(2*l_y)*((y_i - y_j)**2) - 1/(2*l_t)*((t_i - t_j)**2))
kuu_fn = sp.lambdify((x_i, x_j, y_i, y_j, t_i, t_j, sigma, l_x, l_y, l_t), kuu_sym, "numpy")
def kuu(x, y, t, sigma, l_x, l_y, l_t):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kuu_fn(x[i], x[j], y[i], y[j], t[i], t[j], sigma, l_x, l_y, l_t)
    return k

$k_{ff}(X_i,X_j;\theta,\phi) \\
= \mathcal{L}_{X_i}^{\phi} \mathcal{L}_{X_j}^{\phi} k_{uu}(X_i, X_j; \theta) \\
= a^2\partial_{x_i, x_j}k_{uu} + ab \partial_{x_i, y_j}k_{uu} + ac \partial_{x_i, x_j, x_j}k_{uu} + ad \partial_{x_i, y_j, y_j}k_{uu} - a \partial_{x_i, t_j}k_{uu} 
\\+ ba\partial_{y_i, x_j}k_{uu} + b^2\partial_{y_i, y_j}k_{uu} + bc\partial_{y_i, x_j, x_j}k_{uu} + bd\partial_{y_i, y_j, y_j}k_{uu} - b\partial_{y_i, t_j}k_{uu}
\\+ ca\partial_{x_i, x_i, x_j}k_{uu}+ cb\partial_{x_i, x_i, y_j}k_{uu}+ c^2\partial_{x_i, x_i, x_j, x_j}k_{uu}+ cd\partial_{x_i, x_i, y_j, y_j}k_{uu}- c\partial_{x_i, x_i, t_j}k_{uu}
\\+ da\partial_{y_i, y_i,x_j}k_{uu}+ db\partial_{y_i, y_i, y_j}k_{uu}+ dc\partial_{y_i, y_i, x_j, x_j}k_{uu}+ d^2\partial_{y_i, y_i, y_j, y_j}k_{uu}- d\partial_{y_i, y_i, t_j}k_{uu}
\\- a\partial_{t_i, x_j}k_{uu}- b\partial_{t_i, y_j}k_{uu}- c\partial_{t_i, x_j, x_j}k_{uu}- d\partial_{t_i, y_j, y_j}k_{uu}+ \partial_{t_i, t_j}k_{uu}$

In [10]:
kff_sym = a**2*sp.diff(kuu_sym, x_i, x_j) \
        + a*b*sp.diff(kuu_sym, x_i, y_j) \
        + a*c*sp.diff(kuu_sym, x_i, x_j, x_j) \
        + a*d*sp.diff(kuu_sym, x_i, y_j, y_j) \
        - a*sp.diff(kuu_sym, x_i, t_j) \
        + b*a*sp.diff(kuu_sym, y_i, x_j) \
        + b**2*sp.diff(kuu_sym, y_i, y_j) \
        + b*c*sp.diff(kuu_sym, y_i, x_j, x_j) \
        + b*d*sp.diff(kuu_sym, y_i, y_j, y_j) \
        - b*sp.diff(kuu_sym, y_i, t_j) \
        + c*a*sp.diff(kuu_sym, x_i, x_i, x_j) \
        + c*b*sp.diff(kuu_sym, x_i, x_i, y_j) \
        + c**2*sp.diff(kuu_sym, x_i, x_i, x_j, x_j) \
        + c*d*sp.diff(kuu_sym, x_i, x_i, y_j, y_j) \
        - c*sp.diff(kuu_sym, x_i, x_i, t_j) \
        + d*a*sp.diff(kuu_sym, y_i, y_i, x_j) \
        + d*b*sp.diff(kuu_sym, y_i, y_i, y_j) \
        + d*c*sp.diff(kuu_sym, y_i, y_i, x_j, x_j) \
        + d**2*sp.diff(kuu_sym, y_i, y_i, y_j, y_j) \
        - d*sp.diff(kuu_sym, y_i, y_i, t_j) \
        - a*sp.diff(kuu_sym, t_i, x_j) \
        - b*sp.diff(kuu_sym, t_i, y_j) \
        - c*sp.diff(kuu_sym, t_i, x_j, x_j) \
        - d*sp.diff(kuu_sym, t_i, y_j, y_j) \
        + sp.diff(kuu_sym, t_i, t_j)
kff_fn = sp.lambdify((x_i, x_j, y_i, y_j, t_i, t_j, sigma, l_x, l_y, l_t, a,b,c,d), kff_sym, "numpy")
def kff(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kff_fn(x[i], x[j], y[i], y[j], t[i], t[j], sigma, l_x, l_y, l_t, a,b,c,d)
    return k

$k_{fu}(X_i,X_j;\theta,\phi) \\
= \mathcal{L}_{X_i}^{\phi} k_{uu}(X_i, X_j; \theta) \\
= a\partial_{x_i}k_{uu} + b \partial_{y_i}k_{uu} + c \partial_{x_i, x_i}k_{uu} + d \partial_{y_i, y_i}k_{uu} -  \partial_{t_i}k_{uu}$

In [11]:
kfu_sym = a*sp.diff(kuu_sym, x_i) \
        + b*sp.diff(kuu_sym, y_i) \
        + c*sp.diff(kuu_sym, x_i, x_i) \
        + d*sp.diff(kuu_sym, y_i, y_i) \
        - sp.diff(kuu_sym, t_i)
kfu_fn = sp.lambdify((x_i, x_j, y_i, y_j, t_i, t_j, sigma, l_x, l_y, l_t, a,b,c,d), kfu_sym, "numpy")
def kfu(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d):
    k = np.zeros((x.size, x.size))
    for i in range(x.size):
        for j in range(x.size):
            k[i,j] = kfu_fn(x[i], x[j], y[i], y[j], t[i], t[j], sigma, l_x, l_y, l_t, a,b,c,d)
    return k

In [12]:
def kuf(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d):
    return kfu(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d).T

#### Step 3: Compute NLML (with Cholesky decomposition)

Implementing the covariance matrix K and its inverse

In [13]:
def K(sigma, l_x, l_y, l_t, a,b,c,d, s):
    K_mat = np.block([
        [kuu(x, y, t, sigma, l_x, l_y, l_t) + s*np.eye(n), kuf(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d)],
        [kfu(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d), kff(x, y, t, sigma, l_x, l_y, l_t, a,b,c,d) + s*np.eye(n)]
    ])
    return K_mat

In [14]:
def K_inv_and_det(sigma, l_x, l_y, l_t, a,b,c,d, s):
    
    K_inv = np.zeros((2*n, 2*n))
    log_sum = 0
    
    # Use Cholesky, if possible. Otherwise use SVD.
        
    try:
        L = np.linalg.cholesky(K(sigma, l_x, l_y, l_t, a,b,c,d, s))
        L_inv = solve_triangular(L, np.identity(2*n), lower=True) # Slight performance boost over np.linalg.inv
        K_inv = (L_inv.T).dot(L_inv)

        for i in range(2*n):
            log_sum = log_sum + np.log(np.abs(L[i,i]))
    except np.linalg.LinAlgError:
        # Inverse of K via SVD
        u, s_mat, vt = np.linalg.svd(K(sigma, l_x, l_y, l_t, a,b,c,d, s))
        K_inv = (vt.T).dot(np.linalg.inv(np.diag(s_mat))).dot(u.T)  

        # Calculating the log of the determinant of K
        # Singular values are always positive.
        for i in range(s_mat.size):
            log_sum = log_sum + np.log(s_mat[i])
     
    return K_inv, log_sum

Implementing normalized negative log-likelihood function

In [15]:
def nlml(params):

    b_par = 2
    c_par = 0.5
    d_par = 0.5
    
    # Exponentiation to enable unconstrained optimization
    sigma_exp = np.exp(params[0]) 
    l_x_exp = np.exp(params[1])
    l_y_exp = np.exp(params[2]) 
    l_t_exp = np.exp(params[3])
    # a = params[4]
    y_con = np.concatenate((y_u, y_f))
    
    A,b = K_inv_and_det(sigma_exp, l_x_exp, l_y_exp, l_t_exp, params[4],b_par,c_par,d_par, s)
        
    val = b + y_con @ A @ y_con
    return val

#### Step 4: Optimize hyperparameters

**1. Nelder-Mead**

In [16]:
def callbackf(params):
    print(params)

In [17]:
def minimize_restarts(x,y,y_u,y_f,n=5): 
    all_results = []
    for it in range(0,n):
        all_results.append(opt.minimize(nlml, np.random.rand(5), callback = callbackf, method="Nelder-Mead", 
                                        options={'maxfev':5000, 'fatol':0.001, 'xatol':0.001}))
    return min(all_results, key = lambda x: x.fun)

In [18]:
t0 = time.time()
m = minimize_restarts(x, y, y_u, y_f, 3)
t_Nelder = time.time() - t0

[0.58688746 0.77287311 0.48609598 0.6413445  0.70305409]
[0.58688746 0.77287311 0.48609598 0.6413445  0.70305409]
[0.58688746 0.77287311 0.48609598 0.6413445  0.70305409]
[0.62339984 0.75775914 0.45498583 0.63641299 0.67798987]
[0.62339984 0.75775914 0.45498583 0.63641299 0.67798987]
[0.61579834 0.63253309 0.50003664 0.65346105 0.6723021 ]
[0.61579834 0.63253309 0.50003664 0.65346105 0.6723021 ]
[0.6599896  0.5265243  0.45816094 0.7037533  0.70259112]
[0.72071678 0.45428484 0.45536928 0.67245641 0.65699024]
[0.72071678 0.45428484 0.45536928 0.67245641 0.65699024]
[0.73400892 0.12999071 0.49927451 0.72963861 0.70157085]
[0.73400892 0.12999071 0.49927451 0.72963861 0.70157085]
[ 0.87951615 -0.1900617   0.39993537  0.8135771   0.7024854 ]
[ 0.87951615 -0.1900617   0.39993537  0.8135771   0.7024854 ]
[ 0.99281928 -0.61828058  0.45474089  0.87030511  0.65174489]
[ 0.99281928 -0.61828058  0.45474089  0.87030511  0.65174489]
[ 1.1731853  -1.30578558  0.42328827  0.94033984  0.72507837]
[ 1.42

[ 7.29263941 -5.73687882 -1.21294299 -0.84694186 -2.93737531]
[ 7.29263941 -5.73687882 -1.21294299 -0.84694186 -2.93737531]
[ 7.24677216 -5.73931999 -1.20080458 -0.83155472 -2.89052147]
[ 7.24677216 -5.73931999 -1.20080458 -0.83155472 -2.89052147]
[ 7.29346285 -5.71747319 -1.22618155 -0.87091183 -2.90193881]
[ 7.29346285 -5.71747319 -1.22618155 -0.87091183 -2.90193881]
[ 7.29346285 -5.71747319 -1.22618155 -0.87091183 -2.90193881]
[ 7.26756719 -5.72608069 -1.22681287 -0.87359982 -2.84428712]
[ 7.26756719 -5.72608069 -1.22681287 -0.87359982 -2.84428712]
[ 7.26756719 -5.72608069 -1.22681287 -0.87359982 -2.84428712]
[ 7.31677798 -5.70243977 -1.26465131 -0.9294954  -2.82242591]
[ 7.31677798 -5.70243977 -1.26465131 -0.9294954  -2.82242591]
[ 7.24079847 -5.71136661 -1.25522052 -0.92001116 -2.70470039]
[ 7.24079847 -5.71136661 -1.25522052 -0.92001116 -2.70470039]
[ 7.24079847 -5.71136661 -1.25522052 -0.92001116 -2.70470039]
[ 7.34479423 -5.64666094 -1.3440596  -1.05999944 -2.6259519 ]
[ 7.3447

[ 6.64074132 -3.30399926 -2.66794643 -3.45003572  2.00353666]
[ 6.64930648 -3.30393674 -2.67015286 -3.45338181  1.99578829]
[ 6.64698079 -3.3151664  -2.66892612 -3.45039639  2.00250338]
[ 6.66355899 -3.29936471 -2.67901131 -3.46686404  1.99738856]
[ 6.66355899 -3.29936471 -2.67901131 -3.46686404  1.99738856]
[ 6.66355899 -3.29936471 -2.67901131 -3.46686404  1.99738856]
[ 6.64808767 -3.3037466  -2.67130138 -3.45498415  2.00154229]
[ 6.65224837 -3.30369632 -2.67251945 -3.45681743  1.99824351]
[ 6.65224837 -3.30369632 -2.67251945 -3.45681743  1.99824351]
[ 6.65224837 -3.30369632 -2.67251945 -3.45681743  1.99824351]
[ 6.65289018 -3.30811268 -2.6722412  -3.4560515   1.9989703 ]
[ 6.65289018 -3.30811268 -2.6722412  -3.4560515   1.9989703 ]
[ 6.65289018 -3.30811268 -2.6722412  -3.4560515   1.9989703 ]
[ 6.65717259 -3.30258455 -2.67536855 -3.46117336  1.99845789]
[ 6.65717259 -3.30258455 -2.67536855 -3.46117336  1.99845789]
[ 6.65717259 -3.30258455 -2.67536855 -3.46117336  1.99845789]
[ 6.6571

[ 6.87791334 -3.33292457 -2.15911179 -4.0460199   2.06214681]
[ 6.87791334 -3.33292457 -2.15911179 -4.0460199   2.06214681]
[ 6.92916663 -3.29044353 -2.17853818 -4.11047429  2.06147124]
[ 6.92916663 -3.29044353 -2.17853818 -4.11047429  2.06147124]
[ 6.92916663 -3.29044353 -2.17853818 -4.11047429  2.06147124]
[ 6.92916663 -3.29044353 -2.17853818 -4.11047429  2.06147124]
[ 6.92916663 -3.29044353 -2.17853818 -4.11047429  2.06147124]
[ 6.94436366 -3.29839026 -2.17757715 -4.12051458  2.05570807]
[ 6.94436366 -3.29839026 -2.17757715 -4.12051458  2.05570807]
[ 6.98775114 -3.29774298 -2.17203628 -4.179256    2.06719416]
[ 6.98775114 -3.29774298 -2.17203628 -4.179256    2.06719416]
[ 7.00090216 -3.26397887 -2.18277869 -4.19474962  2.05351819]
[ 7.01747725 -3.26818296 -2.19573482 -4.20629829  2.0586219 ]
[ 7.01747725 -3.26818296 -2.19573482 -4.20629829  2.0586219 ]
[ 7.01747725 -3.26818296 -2.19573482 -4.20629829  2.0586219 ]
[ 7.01747725 -3.26818296 -2.19573482 -4.20629829  2.0586219 ]
[ 7.0174

[ 6.82092365 -6.88216636  0.73864604 -4.33786376  1.29241364]
[ 6.86732832 -6.68021787  0.76170306 -5.44772713  0.61022822]
[ 6.86732832 -6.68021787  0.76170306 -5.44772713  0.61022822]
[ 6.86732832 -6.68021787  0.76170306 -5.44772713  0.61022822]
[ 6.86732832 -6.68021787  0.76170306 -5.44772713  0.61022822]
[ 6.86732832 -6.68021787  0.76170306 -5.44772713  0.61022822]
[ 6.92871152 -6.43553672  0.81603952 -6.9966298   0.0619486 ]
[ 6.91941507 -6.45396407  0.80574292 -6.86220052  0.03392346]
[ 6.91941507 -6.45396407  0.80574292 -6.86220052  0.03392346]
[ 6.89850603 -6.2027197   0.8282859  -7.84575897 -0.47161773]
[ 6.89850603 -6.2027197   0.8282859  -7.84575897 -0.47161773]
[ 6.89850603 -6.2027197   0.8282859  -7.84575897 -0.47161773]
[ 6.89850603 -6.2027197   0.8282859  -7.84575897 -0.47161773]
[ 6.89850603 -6.2027197   0.8282859  -7.84575897 -0.47161773]
[ 6.85128969 -6.18434684  0.79838224 -7.52599633 -0.65179857]
[ 6.82900737 -5.98572179  0.81455904 -8.19313239 -1.00718152]
[ 6.8290

...

[ 3.93258453 -4.75725741 -0.9309987  -5.29472953  1.9636176 ]
[ 3.62203088 -4.7806183  -0.95357385 -5.52286113  1.9623366 ]
[ 3.86835703 -4.73892977 -0.92451781 -5.44519655  2.00485061]
[ 3.86835703 -4.73892977 -0.92451781 -5.44519655  2.00485061]
[ 3.87551205 -4.70004598 -0.97133795 -5.41318013  1.97233074]
[ 3.71132066 -4.72753478 -1.03469193 -5.46743795  1.96221152]
[ 3.44877008 -4.72051163 -1.02585739 -5.84920103  1.99984522]
[ 3.44877008 -4.72051163 -1.02585739 -5.84920103  1.99984522]
[ 3.44877008 -4.72051163 -1.02585739 -5.84920103  1.99984522]
[ 3.44877008 -4.72051163 -1.02585739 -5.84920103  1.99984522]
[ 3.23475991 -4.66349027 -1.19229092 -6.04776568  2.0178066 ]
[ 3.23475991 -4.66349027 -1.19229092 -6.04776568  2.0178066 ]
[ 3.23475991 -4.66349027 -1.19229092 -6.04776568  2.0178066 ]
[ 2.48006739 -4.63875606 -1.34172767 -6.70088823  2.01218619]
[ 2.48006739 -4.63875606 -1.34172767 -6.70088823  2.01218619]
[ 2.22343932 -4.53347729 -1.54145869 -7.19186272  2.08115943]
[ 2.2234

[0.47764538 0.17748812 0.50316045 0.21831618 0.71342333]
[0.4816485  0.16485097 0.48202772 0.23115317 0.70400614]
[0.49769739 0.17501749 0.46934807 0.21047426 0.69835583]
[0.49769739 0.17501749 0.46934807 0.21047426 0.69835583]
[0.52711088 0.16272452 0.49283158 0.21996211 0.64257411]
[0.55551646 0.14804475 0.46248255 0.2137889  0.70650734]
[0.58222349 0.14031805 0.41794939 0.22329823 0.64243019]
[0.58222349 0.14031805 0.41794939 0.22329823 0.64243019]
[0.64658248 0.11927657 0.43649181 0.19489749 0.60974666]
[0.74161981 0.08550259 0.41226974 0.21361466 0.55457217]
[0.81149819 0.06375538 0.31696563 0.19083049 0.61333443]
[0.81149819 0.06375538 0.31696563 0.19083049 0.61333443]
[ 0.97638612 -0.01912711  0.2751349   0.20078589  0.46341446]
[ 1.21561869 -0.08436618  0.2372564   0.15199919  0.37290739]
[ 1.21561869 -0.08436618  0.2372564   0.15199919  0.37290739]
[ 1.49392136e+00 -1.90154687e-01  1.26308319e-03  1.29662736e-01
  3.14194278e-01]
[ 1.49392136e+00 -1.90154687e-01  1.26308319e-0

[ 7.9052259  -2.62445205 -3.44384294 -3.44483487  2.19210516]
[ 7.9052259  -2.62445205 -3.44384294 -3.44483487  2.19210516]
[ 7.9052259  -2.62445205 -3.44384294 -3.44483487  2.19210516]
[ 7.9052259  -2.62445205 -3.44384294 -3.44483487  2.19210516]
[ 7.75006281 -2.53359821 -3.5673191  -3.23647183  1.98322828]
[ 7.29170229 -2.32809979 -3.71645636 -3.25921174  2.17846237]
[ 7.29170229 -2.32809979 -3.71645636 -3.25921174  2.17846237]
[ 7.52645673 -2.43295679 -3.63242905 -3.22911721  2.04935921]
[ 7.52645673 -2.43295679 -3.63242905 -3.22911721  2.04935921]
[ 7.16419651 -2.25873446 -3.79442621 -3.1270355   2.06234723]
[ 7.16419651 -2.25873446 -3.79442621 -3.1270355   2.06234723]
[ 7.16419651 -2.25873446 -3.79442621 -3.1270355   2.06234723]
[ 7.16419651 -2.25873446 -3.79442621 -3.1270355   2.06234723]
[ 7.16419651 -2.25873446 -3.79442621 -3.1270355   2.06234723]
[ 7.16419651 -2.25873446 -3.79442621 -3.1270355   2.06234723]
[ 7.16419651 -2.25873446 -3.79442621 -3.1270355   2.06234723]
[ 7.1641

[ 5.8588204  -1.7082849  -3.70889857 -5.59309038  2.01763943]
[ 5.8588204  -1.7082849  -3.70889857 -5.59309038  2.01763943]
[ 5.8588204  -1.7082849  -3.70889857 -5.59309038  2.01763943]
[ 5.78052202 -1.68888827 -3.58489274 -6.22332047  2.0009552 ]
[ 5.78052202 -1.68888827 -3.58489274 -6.22332047  2.0009552 ]
[ 5.78052202 -1.68888827 -3.58489274 -6.22332047  2.0009552 ]
[ 5.78052202 -1.68888827 -3.58489274 -6.22332047  2.0009552 ]
[ 5.78052202 -1.68888827 -3.58489274 -6.22332047  2.0009552 ]
[ 5.73026228 -1.65176275 -3.714047   -5.8169828   1.99755308]
[ 5.47316148 -1.55878273 -3.57497533 -6.85007568  2.00237131]
[ 5.47316148 -1.55878273 -3.57497533 -6.85007568  2.00237131]
[ 5.47316148 -1.55878273 -3.57497533 -6.85007568  2.00237131]
[ 5.21320027 -1.4626618  -3.43708131 -7.85387881  1.9708906 ]
[ 4.98298058 -1.33959647 -3.64276623 -7.52438748  1.97010039]
[ 4.98298058 -1.33959647 -3.64276623 -7.52438748  1.97010039]
[ 4.98298058 -1.33959647 -3.64276623 -7.52438748  1.97010039]
[ 4.9829

[ 4.98723128 -1.33023088 -3.74273253 -7.28888979  1.9821581 ]
[ 4.98723128 -1.33023088 -3.74273253 -7.28888979  1.9821581 ]
[ 4.98723128 -1.33023088 -3.74273253 -7.28888979  1.9821581 ]
[ 4.98988006 -1.33128458 -3.74333287 -7.29017596  1.98173078]
[ 4.98988006 -1.33128458 -3.74333287 -7.29017596  1.98173078]
[ 4.99207313 -1.33214292 -3.74399261 -7.2855244   1.98164163]
[ 4.99207313 -1.33214292 -3.74399261 -7.2855244   1.98164163]
[ 4.99207313 -1.33214292 -3.74399261 -7.2855244   1.98164163]
[ 4.99207313 -1.33214292 -3.74399261 -7.2855244   1.98164163]
[ 4.99207313 -1.33214292 -3.74399261 -7.2855244   1.98164163]
[ 4.99207313 -1.33214292 -3.74399261 -7.2855244   1.98164163]
[ 4.99207313 -1.33214292 -3.74399261 -7.2855244   1.98164163]
[ 4.99675293 -1.33417867 -3.74376516 -7.28341062  1.98163784]
[ 4.99675293 -1.33417867 -3.74376516 -7.28341062  1.98163784]
[ 4.99675293 -1.33417867 -3.74376516 -7.28341062  1.98163784]
[ 4.99675293 -1.33417867 -3.74376516 -7.28341062  1.98163784]
[ 4.9971

In [19]:
t_Nelder

1865.7036004066467

In [20]:
print('The inferred parameter is:', m.x[4])

The inferred parameter is: 2.1634004301602183
